# CoastSat Web: Extracción de información

## Initial Settings

In [26]:
import os
import geopandas as gpd
import pandas as pd

In [56]:
country_info = {
    "country": "CHILE",
    "country_code": "chi",
    "region": {
        "OHIGGINS": {
            "start_code": "0345",
            "end_code": "0362"
        }
    }
}

region_name = "OHIGGINS"  # Región a procesar
input_folder = 'CoastSat_Web'  # Carpeta con los .geojson


## Extracción de información

In [28]:
def convert_geojson_to_excel(input_folder):
    """
    Convierte todos los archivos .geojson de una carpeta a formato .xlsx y los guarda en otra carpeta.

    :param input_folder: Ruta de la carpeta que contiene los archivos .geojson.
    """

    # Iterar sobre todos los archivos en la carpeta de entrada
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.geojson'):
            # Ruta completa del archivo de entrada
            file_path = os.path.join(input_folder, file_name)

            # Cargar el archivo .geojson
            data = gpd.read_file(file_path)

            # Convertir geometría a formato WKT
            data['geometry'] = data['geometry'].apply(lambda x: x.wkt)

            # Ruta completa del archivo de salida
            output_path = os.path.join(input_folder, file_name.replace('.geojson', '.xlsx'))

            # Guardar en formato Excel
            data.to_excel(output_path, index=False)
            print(f"Archivo convertido y guardado en: {output_path}")

    print("Todos los archivos .geojson han sido procesados.")

Se convierten los archivos descargados .geoson a un formato amigable para lectura, excel.

In [29]:
convert_geojson_to_excel(input_folder)

C:\Users\pedro\AppData\Local\Temp\ipykernel_12068\2211421645.py:18: UserWarning: Geometry column does not contain geometry.
  data['geometry'] = data['geometry'].apply(lambda x: x.wkt)


Archivo convertido y guardado en: CoastSat Web\CoastSat_polygon_layer.xlsx


C:\Users\pedro\AppData\Local\Temp\ipykernel_12068\2211421645.py:18: UserWarning: Geometry column does not contain geometry.
  data['geometry'] = data['geometry'].apply(lambda x: x.wkt)


Archivo convertido y guardado en: CoastSat Web\CoastSat_transect_layer.xlsx
Todos los archivos .geojson han sido procesados.


## Extracción de polígonos

In [46]:
def extract_region_polygons(geojson_file, country_info, region_name, region_excel_file):
    """
    Extrae las filas correspondientes a una región de un país en base a un diccionario
    y guarda el resultado en un archivo Excel.

    :param geojson_file: Ruta al archivo .geojson.
    :param country_info: Diccionario con la información del país y las regiones.
    :param region_name: Nombre de la región a filtrar (clave en el diccionario).
    :param region_excel_file: Ruta para guardar el archivo Excel filtrado.
    """
    # Obtener información del país y región
    country_code = country_info["country_code"]
    region_info = country_info["region"][region_name]
    start_code = int(region_info["start_code"])
    end_code = int(region_info["end_code"])

    # Cargar el archivo GeoJSON
    data = gpd.read_file(geojson_file)

    # Crear la carpeta de salida si no existe
    os.makedirs(os.path.join(os.getcwd(), input_folder, region_name), exist_ok=True)
    region_folder = os.path.join(os.getcwd(), input_folder, region_name)
    excel_file = os.path.join(os.getcwd(), region_folder, region_excel_file)


    # Filtrar las filas por el país y el rango de códigos
    filtered_data = data[data['SiteId'].str.startswith(country_code) & 
                         data['SiteId'].str[-4:].astype(int).between(start_code, end_code)]

    # Convertir geometría a formato WKT para exportar a Excel
    filtered_data['geometry'] = filtered_data['geometry'].apply(lambda x: x.wkt)

    # Guardar el resultado en un archivo Excel
    filtered_data.to_excel(excel_file, index=False)
    print(f"Datos filtrados de {region_name} guardados en: {excel_file}")




In [47]:
polygon_file = os.path.join(os.getcwd(), input_folder, 'CoastSat_polygon_layer.geojson')
region_excel_file = 'OHIGGINS_polygons.xlsx'

extract_region_polygons(polygon_file, country_info, region_name, region_excel_file)

Datos filtrados de OHIGGINS guardados en: c:\Users\pedro\OneDrive\Escritorio\RAFA\CoastSat-RRG\CoastSat Web\OHIGGINS\OHIGGINS_polygons.xlsx


C:\Users\pedro\AppData\Local\Temp\ipykernel_12068\3074765093.py:31: UserWarning: Geometry column does not contain geometry.
  filtered_data['geometry'] = filtered_data['geometry'].apply(lambda x: x.wkt)
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [57]:
def save_beach_as_geojson(excel_file):
    """
    Lee un archivo Excel que contiene playas y sus polígonos y guarda cada playa como un archivo .geojson individual.

    :param excel_file: Ruta del archivo Excel con los datos de las playas.
    """
    

    # Crear la carpeta de salida si no existe
    os.makedirs(os.path.join(os.getcwd(), input_folder, region_name), exist_ok=True)
    region_folder = os.path.join(os.getcwd(), input_folder, region_name)
    excel_file = os.path.join(os.getcwd(), region_folder, region_excel_file)
    
    # Cargar el archivo Excel
    data = pd.read_excel(excel_file)
    # Convertir el DataFrame a un GeoDataFrame
    gdf = gpd.GeoDataFrame(data, geometry=gpd.GeoSeries.from_wkt(data['geometry']))


    # Guardar cada fila (playa) como un archivo .geojson individual
    for index, row in gdf.iterrows():
        site_id = row['SiteId']
        geometry = row['geometry']

        # Crear un GeoDataFrame con una sola fila
        playa_gdf = gpd.GeoDataFrame([row], geometry=[geometry])

        # Guardar el archivo .geojson con el nombre del SiteId
        playa_gdf.to_file(region_folder, driver="GeoJSON")
        print(f"Archivo .geojson guardado para {site_id}: {region_folder}")

    print(f"Todos los archivos .geojson han sido guardados en: {input_folder}")

In [58]:
save_beach_as_geojson(region_excel_file)

c:\Users\pedro\OneDrive\Escritorio\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS


c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


PermissionError: [WinError 5] Acceso denegado: 'c:\\Users\\pedro\\OneDrive\\Escritorio\\RAFA\\CoastSat-RRG\\CoastSat_Web\\OHIGGINS'